In [ ]:
# !pip install git+https://github.com/SethPoulsen/sklearn-crfsuite.git@add-missing-attributes#egg=sklearn_crfsuite

In [ ]:
# !pip install --force-reinstall --no-dependencies "scikit-learn==0.24.2"

In [2]:
# import pandas as pd
import json
import sys
import copy


from itertools import chain

import sklearn
from sklearn.metrics import make_scorer

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

### Load data

In [3]:
list_of_stc = []
list_of_list_token_of_stc = []
list_of_list_label_of_stc = []
list_of_list_POStag = []

list_of_sentiment = []
list_of_stc_id = []

with open("./file12345_isPOStag_NER_phase_filter_UNK_oversampling.txt", "r", encoding = "utf-8") as f:
    while f:
        info = f.readline()[:-1] # bỏ dấu xuống dòng
        stc = f.readline()[:-1]
        token = f.readline()[:-1]
        label = f.readline()[:-1]
        postag = f.readline()[:-1]

        # check hết file
        if info == "" and stc == "" and label == "":
            break

        # add idx, add sentiment
        info_split = info.split("\t")
        list_of_stc_id.append(int(info_split[0]))
        sentiment = json.loads(info_split[1])

        # add sentiment
        if sentiment[0] == 0:
            list_of_sentiment.append([1])
        elif sentiment[0] in [1, 2]:
            list_of_sentiment.append([2])
        else:
            assert sentiment[0] in [-1, -2]
            list_of_sentiment.append([0])

        # add stc, add label, add postag
        list_of_stc.append(json.loads(stc))
        list_of_list_token_of_stc.append(json.loads(token))
        list_of_list_label_of_stc.append(json.loads(label))
        list_of_list_POStag.append(json.loads(postag))

In [4]:
# Check how many labels are there in the dataset
unique_labels = set()

for list_lb in list_of_list_label_of_stc:
  [unique_labels.add(lb) for lb in list_lb if lb not in unique_labels]

print(unique_labels)

{'I-obj', 'I-sub', 'B-ap', 'O', 'B-obj', 'B-sub', 'I-ap'}


In [5]:
import random

def shuffle_data(data:list):
    random.shuffle(data)


def build_data_CEE_CRF(list_of_token_stc, list_of_POS_tag, list_CEE_label, is_unify_entity = False, is_shuffle_data = False):
    data_set = []
    CEE_label_dict = {
        "B-sub": "SUB",
        "I-sub": "SUB",
        "B-obj": "OBJ",
        "I-obj": "OBJ",
        "B-ap": "AP",
        "I-ap": "AP",
        "O": "O"
    }

    for index, CEE_label in enumerate(list_CEE_label):
        pos_tag = list_of_POS_tag[index]
        tokens = list_of_token_stc[index]
        
        assert len(pos_tag) == len(tokens)
        assert len(tokens) ==  len(CEE_label)

        list_for_stc = []
        for i, dict_ in enumerate(pos_tag):
            if is_unify_entity:
                list_for_stc.append(tuple([tokens[i], pos_tag[i], CEE_label_dict[CEE_label[i]]]))
            else:
                assert CEE_label[i] in CEE_label_dict.keys()
                list_for_stc.append(tuple([tokens[i], pos_tag[i], CEE_label_dict[i]]))
        data_set.append(list_for_stc)

    if is_shuffle_data:
        shuffle_data(data_set)
    end_train_idx = 9000
    return data_set[:end_train_idx], data_set[end_train_idx:]

data_NER_CRF_train, data_NER_CRF_test  = build_data_CEE_CRF(list_of_list_token_of_stc, list_of_list_POStag, list_of_list_label_of_stc, True)

In [6]:
data_NER_CRF_train[0]

[('Nhìn', 'V', 'O'),
 ('vào', 'E', 'O'),
 ('những', 'L', 'O'),
 ('thông_số', 'N', 'AP'),
 ('kích_thước', 'N', 'AP'),
 ('trên', 'N', 'O'),
 (',', 'CH', 'O'),
 ('VinFast_Lux_SA', 'Np', 'SUB'),
 ('2.0', 'M', 'SUB'),
 ('đang', 'R', 'O'),
 ('nhỉnh', 'V', 'O'),
 ('hơn', 'A', 'O'),
 ('Kia_Sorento', 'Np', 'OBJ'),
 ('về', 'E', 'O'),
 ('mọi', 'L', 'O'),
 ('chỉ_số', 'N', 'O'),
 (',', 'CH', 'O'),
 ('điều', 'N', 'O'),
 ('này', 'P', 'O'),
 ('giúp', 'V', 'O'),
 ('xe', 'N', 'O'),
 ('vạm_vỡ', 'V', 'O'),
 ('hơn', 'R', 'O'),
 ('khi', 'N', 'O'),
 ('nhìn', 'V', 'O'),
 ('từ', 'E', 'O'),
 ('phần', 'N', 'O'),
 ('thân', 'N', 'O')]

### Sentence Feature Representation

In [7]:
def _word2features_pos(sent, i, window):
    word = sent[i][0]
    postag = sent[i][1]
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:3]': postag[:3],
    }

    for w in range(1, window + 1):
        if i > 0:
            word = sent[i - w][0]
            postag = sent[i - w][1]
            features.update({
                '-{}:word.lower()'.format(w): word.lower(),
                '-{}:word.istitle()'.format(w): word.istitle(),
                '-{}:word.isupper()'.format(w): word.isupper(),
                '-{}:postag'.format(w): postag,
                '-{}:postag[:3]'.format(w): postag[:3],
            })
        else:
            features['-{}:BOS'.format(w)] = True

        if i < len(sent) - w:
            word1 = sent[i + w][0]
            postag1 = sent[i + w][1]
            features.update({
                '+{}:word.lower()'.format(w): word1.lower(),
                '+{}:word.istitle()'.format(w): word1.istitle(),
                '+{}:word.isupper()'.format(w): word1.isupper(),
                '+{}:postag'.format(w): postag1,
                '+{}:postag[:3]'.format(w): postag1[:3],
            })
        else:
            features['-{}:EOS'.format(w)] = True

    return features


def _word2features(sent, i, window):
    word = sent[i][0]
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
    }
    for w in range(1, window + 1):
        if i > 0:
            word = sent[i - w][0]
            features.update({
                '-{}:word.lower()'.format(w): word.lower(),
                '-{}:word.istitle()'.format(w): word.istitle(),
                '-{}:word.isupper()'.format(w): word.isupper(),
            })
        else:
            features['-{}:BOS'.format(w)] = True

        if i < len(sent) - w:
            word1 = sent[i + w][0]
            features.update({
                '+{}:word.lower()'.format(w): word1.lower(),
                '+{}:word.istitle()'.format(w): word1.istitle(),
                '+{}:word.isupper()'.format(w): word1.isupper(),
            })
        else:
            features['-{}:EOS'.format(w)] = True
    return features


def word2features(sent, i, is_pos, window):
    if is_pos:
        return _word2features_pos(sent, i, window)
    return _word2features(sent, i, window)

def sent2features(sent, is_pos, window):
    return [word2features(sent, i, is_pos, window) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent, is_pos):
    if is_pos:
        return [token for token, postag, label in sent]
    return [token for token, label in sent]

### Evaluation Metrics

In [18]:
# Convert output to a single element presentation
def get_entity_output_of_CRF(y_test, y_pred, type_element, is_unify_entity = True):
    assert len(y_test) == len(y_pred), 'Độ dài dữ liệu test và pred không bằng nhau'

    test_list, pred_list = [], []
    for i, stc in enumerate(y_test):
        assert len(y_test[i]) ==  len(y_pred[i]), 'Độ dài câu văn test và pred không bằng nhau'
        
        test_stc, pred_stc = [], []
        for k, _ in enumerate(stc):
            test_stc.append(type_element) if y_test[i][k] == type_element else test_stc.append("O")
            pred_stc.append(type_element) if y_pred[i][k] == type_element else pred_stc.append("O")
            
        test_list.append(test_stc)
        pred_list.append(pred_stc)

    return test_list, pred_list

In [ ]:
# Trả về 0 hoặc 1 nếu là exact hoặc binary
# Trả về độ dài nếu là proportional

def get_first_index(stc : list, element_type):
    for index, e in enumerate(stc):
        if e == element_type:
            return index
    return -1


def get_last_index(stc : list, element_type):
    last_index = -1
    for index, e in enumerate(stc):
        if e == element_type:
            last_index = index
    return last_index


def get_span_index(stc : list, element_type):
    return [get_first_index(stc, element_type), get_last_index(stc, element_type)]


def check_ovelapping(stc_test, stc_pred, element_type, evaluate_type): 
    assert evaluate_type in ["exact", "proportional", "binary"], "Wrong type of evaluate"
    assert element_type in [1, "SUB", "AP", "OBJ"], "Wrong type of element"
    assert len(stc_test) == len(stc_pred), 'Kích thước câu văn test và pred không bằng nhau'

    span_test = get_span_index(stc_test, element_type)
    span_pred = get_span_index(stc_pred, element_type)
    
    # index_1, index_2 = get_first_index(stc_test, element_type), get_last_index(stc_test, element_type)
    # index_3, index_4 = get_first_index(stc_pred, element_type), get_last_index(stc_pred, element_type)
    # assert index_1 <= index_2
    # assert index_3 <= index_4
    # if index_1 == -1:
    #     assert index_2 == -1
    # if index_3 == -1:
    #     assert index_4 == -1
    
    # span_test = [index_1, index_2]
    # span_pred = [index_3, index_4]
    
    span_overlapping = range(max(span_test[0], span_pred[0]), min(span_test[-1], span_pred[-1]) + 1)
        
    if evaluate_type == 'exact':
        if len(span_overlapping) == (span_test[1] - span_test[0] + 1) and (span_pred[1] - span_pred[0] + 1) <= (span_test[1] - span_test[0] + 1):
            return 1

    elif evaluate_type == 'proportional':
        if span_test[0] == -1 and span_test[1] == -1 and span_pred[0] ==-1 and span_pred[1] ==-1: # cả hai ông dự đoán là ko có => ko + thêm gì
            return [0, 0]
        
        elif span_test[0] == -1 and span_test[1] == -1 and span_pred[0] !=-1 and span_pred[1] !=-1:  # dự đoán có, mà nhãn ko có => tạm thời sẽ ko bị phạt với proportional
            return [-1, -1]                                            # như việc là gold = [-1, -1], pred = [-1, -1]

        elif len(span_overlapping) > 0:
            return [len(span_overlapping), span_test[1] - span_test[0] + 1]
        
        else:
            return [-1, -1]  # tức là ko có overlap
            
    elif evaluate_type == 'binary':
        if len(span_overlapping) > 0:
            return 1

    else:
        assert False, "No evaluate type found"
    
    return 0


def get_correct_prediction_tuple(tuple_test, tuple_pred, evaluate_type): 
    sub_test, obj_test, aspect_test, sentiment_test = tuple_test
    sub_pred, obj_pred, aspect_pred, sentiment_pred = tuple_pred

    if sentiment_test != sentiment_pred:
        return 0

    if evaluate_type in ["exact", "binary"]:
        a = check_ovelapping(sub_test, sub_pred, "SUB", evaluate_type) # 0 hoặc 1 refer to False or True
        b = check_ovelapping(obj_test, obj_pred, "OBJ", evaluate_type) # 0 hoặc 1
        c = check_ovelapping(aspect_test, aspect_pred, "AP", evaluate_type) # 0 hoặc 1

        # print(a, b, c)
        if a + b + c == 3:
            return 1
        else:
            return 0
    
    elif evaluate_type == "proportional":
        s1, s2  = check_ovelapping(sub_test, sub_pred, "SUB", evaluate_type) # độ dài overlap, độ dài gold
        o1, o2 = check_ovelapping(obj_test, obj_pred, "OBJ", evaluate_type) # 
        a1, a2 = check_ovelapping(aspect_test, aspect_pred, "AP", evaluate_type) # 

        if s1 == -1 or o1 == -1 or a1 == -1:
            return 0
        else:
            return (s1 + o1 + a1) / (s2 + o2 + a2)

    else:
        assert False, "No evaluate type found"






def evaluate_full_tuple(sub_test, sub_pred, obj_test, obj_pred, aspect_test, aspect_pred, sentiment_test, sentiment_pred, evaluate_type): 
    assert len(sub_test) == len(sub_pred)
    assert len(obj_test) == len(obj_pred)
    assert len(aspect_test) == len(aspect_pred)
    assert len(sentiment_test) == len(sentiment_pred)
    assert len(sub_test) == len(obj_test)

    correct = 0
    total = len(sub_test)

    for i in range(len(sub_test)):
        tuple_test = [sub_test[i], obj_test[i], aspect_test[i], sentiment_test[i]]
        tuple_pred = [sub_pred[i], obj_pred[i], aspect_pred[i], sentiment_pred[i]]
        correct += get_correct_prediction_tuple(tuple_test, tuple_pred, evaluate_type)
        # print(correct)

    precision = correct / total
    recall = precision
    f1 = 2*precision*recall / (2* precision)
    print(precision)
    print(f1)








In [35]:
def check_overlap():
    mesure_type = ["exact", "proportional", "binary"]
    evaluate_type = mesure_type[1]

    x = [-1, -1]      # test
    y = [0, 1]    # pred

    set1 = set(range(x[0], x[1] + 1))
    set2 = set(range(y[0], y[1] + 1))
    overlap = set1.intersection(set2)

    len_span_test = len(set1) if -1 not in set1 else 0
    len_span_pred = len(set2) if -1 not in set2 else 0
    len_overlap = len(overlap)

    print(f'Kích thước test: {len_span_test}')
    print(f'Kích thước pred: {len_span_pred}')
    print(f'Kích thước overlap: {len_overlap}')

    if evaluate_type == 'exact':
        if len_span_pred == 0 and len_span_test == 0:
            return 1
        elif len_span_test == len_overlap and len_span_pred == len_overlap:
            return 1
        return 0

    elif evaluate_type == 'proportional':   # trả về [len_overlap, len_gold]
        if len_span_pred == 0 and len_span_test == 0:                   # cả hai ông dự đoán là ko có => ko + thêm gì (tức là tỉ lệ propotional là giữ nguyên không bị giảm đi)
            return [0, 0]
        elif len_span_pred != 0 and len_span_test == 0:                 # dự đoán có, mà nhãn ko có
            print(f'Cũng ko có overlap: {len_overlap}')
            return [-1, -1]
        elif len_overlap > 0:
            return [len_overlap, len_span_test]
        else:
            return [-1, -1]
        

        if x[0] == -1 and x[1] == -1 and y[0] ==-1 and y[1] ==-1:       # cả hai ông dự đoán là ko có => ko + thêm gì (tức là tỉ lệ propotional là giữ nguyên không bị giảm đi)
            return [0, 0]
        
        elif x[0] == -1 and x[1] == -1 and y[0] !=-1 and y[1] !=-1:  # dự đoán có, mà nhãn ko có => tạm thời sẽ ko bị phạt với proportional
            return [-1, -1]                                            # như việc là gold = [-1, -1], pred = [-1, -1]

        elif len(span_overlapping) > 0:
            return [len(span_overlapping), x[1] - x[0] + 1]
        
        else:
            return [-1, -1]  # tức là ko có overlap
            
    elif evaluate_type == 'binary':
        if len(span_overlapping) > 0:
            return 1

    else:
        assert False, "No evaluate type found"
    
    return 0

print(check_overlap())

Kích thước test: 0
Kích thước pred: 2
Kích thước overlap: 0
Cũng ko có overlap: 0
[-1, -1]


In [10]:
for window_size in range(1,4):
    print(f'\n====================== Window_size : {window_size} ======================')
    print('POS-tag:')
    X_train = [sent2features(sent = s, is_pos = True, window = window_size) for s in data_NER_CRF_train]
    y_train = [sent2labels(sent = s) for s in data_NER_CRF_train]
    X_test = [sent2features(sent = s, is_pos = True, window = window_size) for s in data_NER_CRF_test]
    y_test = [sent2labels(sent = s) for s in data_NER_CRF_test]

    crf_model_pos = sklearn_crfsuite.CRF(
        algorithm='lbfgs',
        c1=0.1,
        c2=0.1,
        max_iterations=100,
        all_possible_transitions=False,
    )

    crf_model_pos.fit(X_train, y_train)
    pred = crf_model_pos.predict(X_test)
    # print(metrics.flat_classification_report(y_test, pred, labels=['SUB', 'OBJ', 'AP'], digits=4))


    # ==========================================================================================================
    # print('No POS-tag:')
    # X_train = [sent2features(sent = s, is_pos = False, window = window_size) for s in data_NER_CRF_train]
    # y_train = [sent2labels(sent = s) for s in data_NER_CRF_train]
    # X_test = [sent2features(sent = s, is_pos = False, window = window_size) for s in data_NER_CRF_test]
    # y_test = [sent2labels(sent = s) for s in data_NER_CRF_test]

    # crf_model = sklearn_crfsuite.CRF(
    #     algorithm='lbfgs',
    #     c1=0.1,
    #     c2=0.1,
    #     max_iterations=100,
    #     all_possible_transitions=False,
    # )

    # crf_model.fit(X_train, y_train)
    # pred = crf_model.predict(X_test)
    # print(metrics.flat_classification_report(y_test, pred, labels=['SUB', 'OBJ', 'AP'], digits=4))


====================== Window_size : 3 ======================
POS-tag:
              precision    recall  f1-score   support

         SUB     0.8146    0.7660    0.7895       453
         OBJ     0.7580    0.6939    0.7245       343
          AP     0.6921    0.5784    0.6302       408

   micro avg     0.7595    0.6819    0.7186      1204
   macro avg     0.7549    0.6794    0.7147      1204
weighted avg     0.7569    0.6819    0.7170      1204



/home/hieunv/CEE_using_CRF/cee_venv/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=['SUB', 'OBJ', 'AP'] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [16]:
evaluate_full_tuple(sub_test, sub_pred, obj_test, obj_pred, ap_test, ap_pred, sentiment_test, sentiment_test, "proportional")

In [17]:
sub_test, sub_pred = get_entity_output_of_CRF(y_test, pred, "SUB")
print(sub_test)

[['O', 'O', 'O', 'O', 'O', 'SUB', 'SUB', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'SUB', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'SUB', 'SUB', 'SUB', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'SUB', 'SUB', 'SUB', 'SUB', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'SUB', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'SUB', 'SUB', 'SUB', 'SUB', 'SUB', 'O', 'O', 'O'], ['O', 'SUB', 'SUB', 'SUB', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'SUB', 'SUB', 'SUB', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'SUB', 'SUB', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',